# TP5

## Constantes et modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Question 1

Dans un but de reconstruction tomographique, on peut modéliser l'atténuation du rayonnement qui passe à travers un objet par la loi de Beer-Lambert:
$$I = I_0e^{-\int\mu(x)\text{d}x}$$
Sous forme discrétisée, on a:
$$I = I_0e^{-\sum\mu(x)\Delta x}$$
où $I$  est l'intensité détectée, $I_0$ est l'intensité initiale, $\mu$ est le coefficient d'atténuation linéaire et $x$ est la distance traversée par les rayons. Sous forme linéaire, on a:
$$\ln\bigg(\frac{I_0}{I}\bigg) = \sum\mu(x)\Delta x$$
Pour simplifier le problème, on peut considérer le corps humain comme étant composé d'eau pure. On peut alors déterminer la fraction du signal qui traverse le corps humain en utilisant le coefficient $\mu$ de l'eau. Selon les données XCOM du NIST [1], En considérant des photons ayant une énergie initiale de 60 keV, l'atténuation dans l'eau est de $2,059\times10^{-1}\ \text{cm}^2/g$. Pour une masse volumique de $1\ \text{g/cm}^3$, on obtient le paramètre $\mu(x)$ suivant:
$$\mu(x) = 2,059\times10^{-1}\ \text{cm}^2/g \times 1\ \text{g/cm}^3 = 2,059\times10^{-1}\ \text{cm}^{-1}$$
On peut alors déterminer la fraction du signal qui va être mesurée après avoir parcouru une distance donnée dans l'eau. Pour une distance de 5 cm, on a
$$\frac{I}{I_0} = e^{-\int \mu(x)\\text{d}x}$$
$$\frac{I}{I_0} = e^{-\int_0^5 2,059\times10^{-1}\text{d}x}$$
$$\frac{I}{I_0} = 0,3572$$
On observera donc 35,8 % de l'intensité du faisceau incident pour une distance de 5 cm dans l'eau (ou le corps humain). Pour une distance de 20 cm, on a:
$$\frac{I}{I_0} = e^{-\int_0^20 2,059\times10^{-1}\text{d}x}$$
$$\frac{I}{I_0} = 0,0163$$
On mesurera donc 1,63 % de l'intensité du faisceau initial pour une distance de 20 cm parcourue dans l'eau.